<a href="https://colab.research.google.com/github/shiffa-04/LangChain/blob/main/RAG_project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install langchain

In [ ]:
pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-chroma

In [ ]:
pip install -qU langchain-openai

In [ ]:
pip install python-docx

In [ ]:
pip install langgraph

In [6]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

··········


In [13]:
from docx import Document
from langchain.schema import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.tools.retriever import create_retriever_tool
from langchain.tools import Tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain.tools.retriever import create_retriever_tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain.schema import HumanMessage

In [12]:
def read_docx(file_path):
  doc = Document(file_path)
  # print(doc)
  text = []
  for para in doc.paragraphs:
    # print(para.text)
    text.append(para.text)
  return '/n'.join(text)

doc_text = read_docx("/content/story2.docx")

In [14]:
docx_text = Document(page_content = doc_text)

In [15]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 200)
chunks = text_splitter.split_documents([docx_text])
print(len(chunks))

11


In [16]:
for i, chunk in enumerate(chunks):
  print(f"Chunk {i+1}:")
  print(chunk)

Chunk 1:
page_content='Kabir, Aryan, Nick, Hamza, and Zayn set out early in the morning for their much- awaited trekking adventure, each one excited to escape the hustle of city life and embrace the beauty of nature. Zayn, the dependable leader, took charge of mapping out the trail and ensuring everyone was prepared for the journey ahead. Nick, with his endless jokes and cheerful spirit, kept the group laughing as they walked, turning even the steepest climbs into fun challenges. Aryan, the energetic fitness'
Chunk 2:
page_content='for the journey ahead. Nick, with his endless jokes and cheerful spirit, kept the group laughing as they walked, turning even the steepest climbs into fun challenges. Aryan, the energetic fitness enthusiast, led the pack with swift strides, encouraging others to keep up and even suggesting quick workout stops to stretch their muscles and keep the blood flowing. Hamza, the talented photographer, lagged slightly behind, capturing the stunning landscapes and ca

In [17]:
vector_store = Chroma.from_documents(documents = chunks, embedding = OpenAIEmbeddings())
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

In [18]:
memory = MemorySaver()

In [19]:
retriever_tool = create_retriever_tool(
    retriever,
    "DocumentRetriever",
    "A tool to retrieve relevant chunks of documents based on a query."
)
tools = [retriever_tool]

In [20]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [30]:
config = {"configurable": {"thread_id": "abcd"}}

In [31]:
for s in agent_executor.stream(
        {"messages":[HumanMessage(content="Hi! I'am Shiffa")]},
        config = config,

):
  print(s)
  print("-----")

{'agent': {'messages': [AIMessage(content='Hello Shiffa! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 70, 'total_tokens': 83}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e92ab49b-4728-412f-a035-036643430553-0', usage_metadata={'input_tokens': 70, 'output_tokens': 13, 'total_tokens': 83})]}}
-----


In [32]:
for s in agent_executor.stream(
        {"messages":[HumanMessage(content="Who was Zayn?")]},
        config = config,

):
  print(s)
  print("-----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_OSBslhqpNWSKjYFT2YkCxeTr', 'function': {'arguments': '{"query":"Zayn"}', 'name': 'DocumentRetriever'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 96, 'total_tokens': 114}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d7d32f64-bc14-49f0-ab88-563698360f6b-0', tool_calls=[{'name': 'DocumentRetriever', 'args': {'query': 'Zayn'}, 'id': 'call_OSBslhqpNWSKjYFT2YkCxeTr', 'type': 'tool_call'}], usage_metadata={'input_tokens': 96, 'output_tokens': 18, 'total_tokens': 114})]}}
-----
{'tools': {'messages': [ToolMessage(content='Kabir, Aryan, Nick, Hamza, and Zayn set out early in the morning for their much- awaited trekking adventure, each one excited to escape the hustle of city life and embrace the beauty of nature. Zayn, the dependable leader, took charge

In [33]:
for s in agent_executor.stream(
        {"messages":[HumanMessage(content="What did he do during the rain?")]},
        config = config,

):
  print(s)
  print("-----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_HUFrVIl9sY7RHhFm5oWeKiuW', 'function': {'arguments': '{"query":"Zayn during the rain"}', 'name': 'DocumentRetriever'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 321, 'total_tokens': 342}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-cd5cbce3-5343-46a5-af53-405d66771d43-0', tool_calls=[{'name': 'DocumentRetriever', 'args': {'query': 'Zayn during the rain'}, 'id': 'call_HUFrVIl9sY7RHhFm5oWeKiuW', 'type': 'tool_call'}], usage_metadata={'input_tokens': 321, 'output_tokens': 21, 'total_tokens': 342})]}}
-----
{'tools': {'messages': [ToolMessage(content="the most raindrops in their hands. Zayn ensured the fire stayed alive despite the drizzle, while Hamza took creative, rain-soaked photos that captured the group's carefree happiness./nAs the rain subs

In [38]:
for s in agent_executor.stream(
        {"messages":[HumanMessage(content="What happened the next morning?")]},
        config = config,

):
  print(s)
  print("-----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8U7UPvH4LxWeIw4lj1doXgnA', 'function': {'arguments': '{"query":"next morning"}', 'name': 'DocumentRetriever'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 1237, 'total_tokens': 1254}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-889f64a3-e77e-47de-9e52-e182dc533927-0', tool_calls=[{'name': 'DocumentRetriever', 'args': {'query': 'next morning'}, 'id': 'call_8U7UPvH4LxWeIw4lj1doXgnA', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1237, 'output_tokens': 17, 'total_tokens': 1254})]}}
-----
{'tools': {'messages': [ToolMessage(content='fascinating stories about explorers who had braved similar terrains. The next morning, they woke up to a breathtaking sunrise, feeling closer than ever after the shared experiences of the previous day. Packing up

In [40]:
for s in agent_executor.stream(
        {"messages":[HumanMessage(content="Who suggested plans for the group's next adventure?")]},
        config = config,

):
  print(s)
  print("-----")

{'agent': {'messages': [AIMessage(content="Kabir suggested plans for the group's next adventure. He shared stories about explorers who had braved similar terrains, inspiring discussions about future adventures. Kabir's enthusiasm for exploration likely sparked excitement within the group for their upcoming endeavors. If you have any more questions or need additional details, feel free to let me know!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 1581, 'total_tokens': 1649}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2cf16799-c078-4016-aac9-ce160b67ec1e-0', usage_metadata={'input_tokens': 1581, 'output_tokens': 68, 'total_tokens': 1649})]}}
-----


In [39]:
for s in agent_executor.stream(
        {"messages":[HumanMessage(content="Who was the funny character?")]},
        config = config,

):
  print(s)
  print("-----")

{'agent': {'messages': [AIMessage(content="Nick was the funny character in the group. He kept everyone laughing with his endless jokes and cheerful spirit during their trekking adventure. Nick's humor added a lighthearted and fun element to the journey, making it enjoyable for the group. If you have any more questions or need further information, feel free to ask!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 1497, 'total_tokens': 1564}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5f627dc7-98bd-4bf2-823c-8c6b59ff1268-0', usage_metadata={'input_tokens': 1497, 'output_tokens': 67, 'total_tokens': 1564})]}}
-----
